In [5]:
import json

from enum import Enum

class Mark(Enum):
    GREEN = 0
    ORANGE = 5
    RED = 10

In [6]:
class Interval:
    def __init__(self, left: float, right: float):
        if not isinstance(left, float) or not isinstance(right, float):
            raise TypeError("Левая и правая границы должны быть вещественными числами.")
        if left > right:
            raise ValueError("Левая граница должна быть меньше или равна правой границе.")
        self.left = float(left)
        self.right = float(right)

    # Проверка пересечения двух интервалов
    def intersects(self, other):
        return self.left <= other.right and other.left <= self.right

    # Проверка включения одного интервала в другой
    def contains_interval(self, other):
        return self.left <= other.left and self.right >= other.right

    # Проверка совпадения двух интервалов
    def __eq__(self, other):
        return self.left == other.left and self.right == other.right

    # Проверка принадлежности числа интервалу
    def contains_value(self, value):
        return self.left <= value <= self.right

    # Вычисление длины интервала
    def length(self):
        return self.right - self.left

    # Получение нижней границы интервала
    def get_left(self):
        return self.left

    # Получение верхней границы интервала
    def get_right(self):
        return self.right

In [7]:
with open("response.json", "r", encoding="utf-8") as file:
        json_file = json.loads(file.read())

In [8]:
def find_meta_value(data, meta):
  """
  Находит первый объект с заданным значением ключа "meta".

  Args:
    data: Данные в формате словаря или списка.
    meta: Искомое значение ключа "meta".

  Returns:
    Словарь с найденным объектом или None, если объект не найден.
  """
  if isinstance(data, dict):
    if 'meta' in data and data['meta'] == meta:
      return data
    for value in data.values():
      result = find_meta_value(value, meta)
      if result:
        return result
  elif isinstance(data, list):
    for item in data:
      result = find_meta_value(item, meta)
      if result:
        return result
  return None

In [9]:

def find_name_value(data, name):
    """
     Function to find an object with a given key "name" and its value.

     Args:
     data: Data in dictionary format.
     name: The searched value of the key "name".

     Returns:
     A dictionary with the found object or None if the object is not found.
    """

    if 'name' in data and data['name'] == name:
        return data

    for item in data.values():
        if isinstance(item, dict):
            result = find_name_value(item, name)
            if result:
                return result
        elif isinstance(item, list):
            for sub_item in item:
                result = find_name_value(sub_item, name)
                if result:
                    return result

    return None

In [10]:
def find_nested_element(data, key1, value1, key2, value2):
    def find_first_pair(data, key1, value1):
        if isinstance(data, dict):
            if key1 in data and data[key1] == value1:
                return data
            for key, value in data.items():
                if isinstance(value, dict):
                    result = find_first_pair(value, key1, value1)
                    if result is not None:
                        return result
                elif isinstance(value, list):
                    for item in value:
                        if isinstance(item, dict):
                            result = find_first_pair(item, key1, value1)
                            if result is not None:
                                return result
        elif isinstance(data, list):
            for item in data:
                result = find_first_pair(item, key1, value1)
                if result is not None:
                    return result
        return None

    first_pair_dict = find_first_pair(data, key1, value1)

    if first_pair_dict:
        return find_first_pair(first_pair_dict, key2, value2)

    return None

In [15]:
result_1 = find_name_value( json_file,"Газы на основе Гелия")
print(result_1)
result_2 = find_name_value( json_file,"Газы на основе Аргона")
print(result_2)

{'id': 17940078395424, 'name': 'Газы на основе Гелия', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Класс газов', 'successors': [{'id': 17940078395400, 'name': 'Гелий', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Газ', 'successors': [{'id': 17940078395406, 'value': 'He', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': 'Химическое обозначение'}, {'id': 17940078395410, 'value': 'высший', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': 'Сорт'}, {'id': 17940078395414, 'value': 'A', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': 'Марка'}]}]}
{'id': 17940078395428, 'name': 'Газы на основе Аргона', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Класс газов', 'successors': [{'id': 17940078395416, 'name': 'Аргон', 'type': 'НЕТЕРМИНАЛ', 'meta': 'Газ', 'successors': [{'id': 17940078395422, 'value': 'Ar', 'type': 'ТЕРМИНАЛ-ЗНАЧЕНИЕ', 'valtype': 'STRING', 'meta': 'Химическое обозначение'}]}]}


In [19]:
def monogas_sim(json1,json2):
    gas_class1 = find_meta_value(json1,"Класс газов")
    gas_class2 = find_meta_value(json2,"Класс газов")

    
    if json1 is None and  json2 is not None:
        return Mark.RED 

    elif json1 is None and  json2 is  None:
        return Mark.GREEN
    
    #1.у газов разные классы
    if gas_class1["name"] != gas_class2["name"]:
        return Mark.RED
    else: 
        gas_name1 = find_meta_value(gas_class1,'Газ')
        gas_name2 = find_meta_value(gas_class2,'Газ')

        #2.у газов не одинаковые имена
        if gas_name1["name"] != gas_name2["name"]:
            return Mark.RED
#3. Имеют одинаковые названия
        else:
            gas_sort1 = find_meta_value(gas_name1,'Сорт')
            gas_sort2 = find_meta_value(gas_name2,'Сорт')

            gas_mark1 = find_meta_value( gas_name1,'Марка')
            gas_mark2 = find_meta_value( gas_name2,'Марка')

            #3.1 Есть и сорт и марка
            if gas_sort1 is not None and gas_sort2 is not None and gas_mark1 is not None and gas_mark2 is not None: 
                #3.1.1
                if gas_sort1 == gas_sort2 and gas_mark1 == gas_mark2:
                    return Mark.GREEN
                #3.1.2
                else:
                    return Mark.ORANGE
            #3.2 Есть неполный набор о сорте и марке газов
            else:
                #3.2.1 
                #есть сорт газов
                if gas_sort1 is not None and gas_sort2 is not None:
                    if gas_sort1 == gas_sort2:
                        return Mark.GREEN
                    else:
                        return Mark.ORANGE
                
                #есть марки газа
                elif gas_mark1 is not None and gas_mark2 is not None:
                    if gas_mark1 == gas_mark2:
                        return Mark.GREEN
                    else:
                        return Mark.ORANGE 
                
                    #3.2.2.если данные в разных пармаетрах  
                elif  (gas_sort1 and gas_mark2 ) or  (gas_mark1 and gas_sort2):
                    return Mark.ORANGE
              #3.3
                elif (gas_sort1 and gas_mark1) is not None != (gas_sort2 and gas_mark2) is not None:
                    if gas_sort1 == gas_sort2 or gas_mark1 == gas_mark2:
                        return Mark.GREEN
                    else:
                        return Mark.ORANGE
                #3.4 Если для одного из моногазов не указаны его сорт и марка, а для другого указаны, то пара отмечается как оранжевая.
                elif ((gas_sort1 and gas_mark1) is not None) != ((gas_sort2 and gas_mark2) is not None):
                    return Mark.ORANGE
                #3.5. Если ни для одного из моногазов не указаны их сорт и марка, то пара отмечается как зелёная.
                else:
                    return  Mark.GREEN
                
            
                    
            

In [23]:
Ocenka =  monogas_sim(result_1,result_2)
print("Ocenka",Ocenka)

Ocenka Mark.RED
